<a href="https://colab.research.google.com/github/Kingsyel/Embeddings_Text_Image_Docs/blob/main/Training_Embedings_using_bert_from_tfhub_and_fine_tuned_one_on_IMDB_moview_review_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade tensorflow==2.13.0 tensorflow-hub==0.13.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-hub
    Found existing installation: tensorflow-hub 0.16.1
    Uninstalling tensorflow-hub-0.16.1:
      Successfully uninstalled tensorflow-hub-0.16.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.13.1
    Uninstalling tensorflow-2.13.1:
      Successfully uninstalled tensorflow-2.13.1


In [2]:
!pip install tensorflow-text==2.13.0

In [3]:

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

# Clear TensorFlow cache to avoid compatibility issues
tf.keras.backend.clear_session()


In [7]:


#  Define the BERT model link from TensorFlow Hub
tfhub_link = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

#  Define the custom classifier using the BERT model
class Classifier(tf.keras.Model):
    def __init__(self, num_classes):
        super(Classifier, self).__init__(name="prediction")
        # Load the BERT encoder as a Keras layer
        self.encoder = hub.KerasLayer(tfhub_link, trainable=True)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.dense = tf.keras.layers.Dense(num_classes)

    def call(self, preprocessed_text):
        # Pass the preprocessed text through the BERT encoder
        encoder_outputs = self.encoder(preprocessed_text)
        # Extract the pooled output (i.e., the [CLS] token's representation)
        pooled_output = encoder_outputs["pooled_output"]

        # Apply dropout for regularization
        x = self.dropout(pooled_output)

        # Pass through the dense layer to get the final predictions
        x = self.dense(x)

        return x

# Initialize the model (e.g., for binary classification)
num_classes = 2
model = Classifier(num_classes)

# Define a BERT-compatible tokenizer
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

def preprocess_text(text):
    return bert_preprocess(tf.constant(text))

# Example usage of the model
sample_text = ["I used up all my money!", "This movie was great!"]
preprocessed_input = preprocess_text(sample_text)

# Run a forward pass to get the predictions
logits = model(preprocessed_input)

# Output the logits (before applying activation like softmax for classification)
print("Logits:", logits)


Logits: tf.Tensor(
[[-0.7703878  -0.20041816]
 [-0.89665145 -0.26106405]], shape=(2, 2), dtype=float32)


In [8]:
# converting logits to probabilities using softmax
predictions = tf.nn.softmax(logits, axis=1)

# Converting probabilities to class labels (0 or 1)
predicted_classes = tf.argmax(predictions, axis=1)

print("Predicted Probabilities:", predictions.numpy())
print("Predicted Classes:", predicted_classes.numpy())


Predicted Probabilities: [[0.3612438  0.63875616]
 [0.34624472 0.65375537]]
Predicted Classes: [1 1]


# Being not satisfied with the output i will fine tune the odel using imdb **review** *datasets*

In [9]:
!pip install tensorflow transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu121 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [10]:
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, create_optimizer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

In [11]:
# Load the IMDb dataset
dataset = load_dataset("imdb")

print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [12]:
# Initialize the tokenizer for BERT
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=128)

# Apply tokenization to the dataset
encoded_dataset = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [13]:
# Prepare data for TensorFlow
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

# Convert dataset to TensorFlow format
tf_train_dataset = encoded_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols="label",
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)

tf_test_dataset = encoded_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    label_cols="label",
    shuffle=False,
    batch_size=32,
    collate_fn=data_collator
)


In [14]:
# Load a pre-trained BERT model for sequence classification
model2 = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [18]:
# Create optimizer with a learning rate schedule
num_train_steps = len(tf_train_dataset) * 3  # for 3 epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5, num_warmup_steps=0, num_train_steps=num_train_steps
)

model2.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])


In [ ]:
# Fine-tune the model
history = model2.fit(
    tf_train_dataset,
    validation_data=tf_test_dataset,
    epochs=3
)


Epoch 1/3


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
 18/782 [..............................] - ETA: 4:21:49 - loss: 0.6825 - accuracy: 0.5677

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(tf_test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")


In [ ]:
# Example sentences for prediction
new_texts = ["I used up all my money!", "This movie was great!"]
inputs = tokenizer(new_texts, return_tensors="tf", padding=True, truncation=True)
predictions = model(inputs)

# Convert logits to probabilities
probs = tf.nn.softmax(predictions.logits, axis=1)
predicted_classes = tf.argmax(probs, axis=1)

print("Predicted classes:", predicted_classes.numpy())
print("Probabilities:", probs.numpy())
